### Environment Setup

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, Tool


e:\Private\AI-PROJECTS-PORTFOLIO-DOCS-ASSETS\ALL-PROJECTS-PACKAGES\gen-ai-and-llm\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

### Create Function Tools

#### Calculator Tool

In [3]:
#Calculator Tool
def calculator_tool(expression: str) -> str:
    try:
        result = eval(expression)
        return f"The result of {expression} is {result}"
    except Exception as e:
        return f"Error: {str(e)}"


#### Weather API Tool

In [12]:
# Weather API

import requests

def get_weather(city: str) -> str:
    city_coords = {
        "pune": (18.5204, 73.8567),
        "mumbai": (19.0760, 72.8777),
        "delhi": (28.6139, 77.2090),
        "malegaon": (29.487, 71.000)
    }
    if city.lower() not in city_coords:
        return "City not found in database."
    
    lat, lon = city_coords[city.lower()]
    url = f"https://api.open-meteo.com/v1/forecast?latitude={lat}&longitude={lon}&current_weather=true"
    res = requests.get(url).json()
    temp = res["current_weather"]["temperature"]
    return f"The current temperature in {city.title()} is {temp}°C."


### Wrap Tools for LangChain Agent

In [13]:
tools = [
    Tool(
        name="Calculator",
        func=calculator_tool,
        description="Useful for mathematical calculations."
    ),
    Tool(
        name="Weather Info",
        func=get_weather,
        description="Provides real-time weather for a given city."
    )
]


### Create Gemini-Powered Agent

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.3)

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type="zero-shot-react-description",
    verbose=True
)

### Test Your AI Assistant Agent

In [15]:
agent.invoke("What is 45 * 23?")
agent.invoke("What's the weather in Mumbai?")
agent.invoke("If temperature in Mumbai is 30°C, what would be in Fahrenheit?")



> Entering new AgentExecutor chain...
I need to calculate the product of 45 and 23.
Action: Calculator
Action Input: 45 * 23
Observation: The result of 45 * 23 is 1035
Thought:I now know the final answer
Final Answer: 1035

> Finished chain.


> Entering new AgentExecutor chain...
I need to find out the current weather conditions in Mumbai.
Action: Weather Info
Action Input: Mumbai
Observation: The current temperature in Mumbai is 29.1°C.
Thought:I now know the weather in Mumbai.
Final Answer: The current temperature in Mumbai is 29.1°C.

> Finished chain.


> Entering new AgentExecutor chain...
I need to convert Celsius to Fahrenheit. I can use the formula: Fahrenheit = (Celsius * 9/5) + 32.
Action: Calculator
Action Input: (30 * 9/5) + 32
Observation: The result of (30 * 9/5) + 32 is 86.0
Thought:I now know the final answer
Final Answer: 86°F

> Finished chain.


{'input': 'If temperature in Mumbai is 30°C, what would be in Fahrenheit?',
 'output': '86°F'}

In [16]:
agent.invoke("What's the weather in Malegaon?")



> Entering new AgentExecutor chain...
I need to find out the weather in Malegaon.
Action: Weather Info
Action Input: Malegaon
Observation: The current temperature in Malegaon is 28.0°C.
Thought:I now know the weather in Malegaon.
Final Answer: The current temperature in Malegaon is 28.0°C.

> Finished chain.


{'input': "What's the weather in Malegaon?",
 'output': 'The current temperature in Malegaon is 28.0°C.'}

In [20]:
agent.invoke("What is the division of 4/0")



> Entering new AgentExecutor chain...
I need to calculate the result of 4 divided by 0. Division by zero is undefined in mathematics.
Action: Calculator
Action Input: 4/0
Observation: Error: division by zero
Thought:I know that division by zero is undefined.
Final Answer: Division by zero is undefined.

> Finished chain.


{'input': 'What is the division of 4/0',
 'output': 'Division by zero is undefined.'}